# First Model: CNN

## Import packages

In [1]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import itertools
import glob 
import seaborn as sns
import tensorflow as tf
import multiprocessing as mp
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Lambda
from tensorflow.keras.models import Model
from astropy.stats import sigma_clip
from tqdm import tqdm
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor, as_completed

sns.set_theme(style='dark')
palette = sns.color_palette('muted')
pd.set_option('display.max_columns', None)

2025-09-30 03:58:16.582752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759204696.790062      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759204696.849907      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load and calibrate the data

In [2]:
path_folder = '/kaggle/input/ariel-data-challenge-2025/' 
path_out = '/kaggle/tmp/data_light_raw/'

if not os.path.exists(path_out):
    os.makedirs(path_out)
    print(f"Directory {path_out} created.")
else:
    print(f"Directory {path_out} already exists.")

CHUNKS_SIZE = 4

Directory /kaggle/tmp/data_light_raw/ created.


In [3]:
def ADC_convert(signal, gain=0.4369, offset=-1000):
    """The Analog-to-Digital Conversion (adc) is performed by the detector to convert
    the pixel voltage into an integer number. Since we are using the same conversion number 
    this year, we have simply hard-coded it inside."""
    signal = signal.astype(np.float64)
    signal /= gain
    signal += offset
    return signal

def mask_hot_dead(signal, dead, dark):
    hot = sigma_clip(
        dark, sigma=5, maxiters=5
    ).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))
    signal = np.ma.masked_where(dead, signal)
    signal = np.ma.masked_where(hot, signal)
    return signal

def apply_linear_corr(linear_corr, clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

def clean_dark(signal, dead, dark, dt):
    dark = np.ma.masked_where(dead, dark)
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark * dt[:, np.newaxis, np.newaxis]
    return signal

def get_cds(signal):
    cds = signal[:,1::2,:,:] - signal[:,::2,:,:]
    return cds

def bin_obs(cds_signal, binning):
    cds_transposed = cds_signal.transpose(0,1,3,2)
    cds_binned = np.zeros((cds_transposed.shape[0], cds_transposed.shape[1] // binning, cds_transposed.shape[2], cds_transposed.shape[3]))
    for i in range(cds_transposed.shape[1] // binning):
        cds_binned[:,i,:,:] = np.sum(cds_transposed[:,i*binning:(i+1)*binning,:,:], axis=1)
    return cds_binned

def correct_flat_field(flat, dead, signal):
    flat = flat.transpose(1, 0)
    dead = dead.transpose(1, 0)
    flat = np.ma.masked_where(dead, flat)
    flat = np.tile(flat, (signal.shape[0], 1, 1))
    signal /= flat
    return signal

def get_index(files, CHUNKS_SIZE):
    index = []
    for file in files:
        file_name = file.split('/')[-1]
        if file_name.split('_')[0] == 'AIRS-CH0' and file_name.split('_')[1] == 'signal' and file_name.split('_')[2] == '0.parquet':
            file_index = os.path.basename(os.path.dirname(file))
            index.append(int(file_index))
    index = np.array(index)
    index = np.sort(index) 
    # credit to DennisSakva
    index = np.array_split(index, len(index) // CHUNKS_SIZE)
    
    return index

In [4]:
files = glob.glob(os.path.join(path_folder + 'train/', '*/*'))
index = get_index(files, CHUNKS_SIZE)

In [5]:
axis_info = pd.read_parquet(os.path.join(path_folder,'axis_info.parquet'))
DO_MASK = False
DO_THE_NL_CORR = False
DO_DARK = False
DO_FLAT = False
TIME_BINNING = True

cut_inf, cut_sup = 39, 321
l = cut_sup - cut_inf

In [6]:
def load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, dataset):
    """Load all calibration data for the chunk at once"""
    calibration_data = {}
    
    for idx in index_chunk:
        calibration_data[idx] = {}
        
        # AIRS calibration data
        airs_flat = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/flat.parquet'))
        calibration_data[idx]['airs_flat'] = airs_flat.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_dark = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/dark.parquet'))
        calibration_data[idx]['airs_dark'] = airs_dark.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_dead = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/dead.parquet'))
        calibration_data[idx]['airs_dead'] = airs_dead.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_linear = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/linear_corr.parquet'))
        calibration_data[idx]['airs_linear'] = airs_linear.values.astype(np.float32).reshape((6, 32, 356))[:, :, cut_inf:cut_sup]
    
    return calibration_data

In [7]:
def process_single_observation(args):
    """Process a single observation with all AIRS and FGS1 cleaning steps"""
    # Unpack the arguments
    (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK) = args
    
    idx = index_chunk[i]
    
    # AIRS Processing
    # Load AIRS signal data
    df = pd.read_parquet(os.path.join(path_folder, f'train/{idx}/AIRS-CH0_signal_0.parquet'))
    signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 356))

    # 1. ADC Conversion
    signal = ADC_convert(signal)
    dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
    dt_airs[1::2] += 0.1
    chopped_signal = signal[:, :, cut_inf:cut_sup]
    del signal, df
    
    # Get pre-loaded calibration data for AIRS
    flat = calibration_data[idx]['airs_flat']
    dark = calibration_data[idx]['airs_dark'] 
    dead_airs = calibration_data[idx]['airs_dead']
    linear_corr = calibration_data[idx]['airs_linear']

    # 2. Mask Hot/Dead Pixels
    if DO_MASK:
        chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_signal)
        chopped_signal = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_signal, dead_airs, dark, dt_airs)
        chopped_signal = cleaned_signal

    # Store AIRS result
    airs_result = chopped_signal
    
    # Return the processed results
    return i, airs_result

In [8]:
def process_single_observation_test(args):
    """Process a single observation with all AIRS and FGS1 cleaning steps"""
    # Unpack the arguments
    (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK) = args
    
    idx = index_chunk[i]
    
    # AIRS Processing
    # Load AIRS signal data
    df = pd.read_parquet(os.path.join(path_folder, f'test/{idx}/AIRS-CH0_signal_0.parquet'))
    signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 356))

    # 1. ADC Conversion
    signal = ADC_convert(signal)
    dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
    dt_airs[1::2] += 0.1
    chopped_signal = signal[:, :, cut_inf:cut_sup]
    del signal, df
    
    # Get pre-loaded calibration data for AIRS
    flat = calibration_data[idx]['airs_flat']
    dark = calibration_data[idx]['airs_dark'] 
    dead_airs = calibration_data[idx]['airs_dead']
    linear_corr = calibration_data[idx]['airs_linear']

    # 2. Mask Hot/Dead Pixels
    if DO_MASK:
        chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_signal)
        chopped_signal = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_signal, dead_airs, dark, dt_airs)
        chopped_signal = cleaned_signal

    # Store AIRS result
    airs_result = chopped_signal
    
    # Return the processed results
    return i, airs_result

In [9]:
for n, index_chunk in enumerate(tqdm(index)):
    # Load all calibration data once at the beginning 
    calibration_data = load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, 'train')
    
    # Pre-allocate output arrays
    AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l), dtype=np.float32)
    
    # Parallel Processing
    # Determine number of workers 
    num_workers = min(2, CHUNKS_SIZE)
    
    # Prepare arguments for each observation
    args_list = []
    for i in range(CHUNKS_SIZE):
        args = (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK)
        args_list.append(args)
        
    # Process observations in parallel
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:        
        # Submit all tasks to the thread pool
        future_to_index = {executor.submit(process_single_observation, args): args[0] for args in args_list}
                
        # Collect results as they complete
        for future in tqdm(as_completed(future_to_index), total=CHUNKS_SIZE, desc=f"Processing observations"):
            i, airs_result = future.result()
            results.append((i, airs_result))
        
    # Sort results by observation index (i) to maintain order
    results.sort(key=lambda x: x[0])
    
    # Store results in existing arrays
    for result in results:
        i, airs_result= result
        AIRS_CH0_clean[i] = airs_result
        
    # 5. Get Correlated Double Sampling
    AIRS_cds = get_cds(AIRS_CH0_clean)
    
    del AIRS_CH0_clean
    
    # 6. (Optional) Time Binning (to reduce space)
    if TIME_BINNING:
        AIRS_cds_binned = bin_obs(AIRS_cds, binning=30)
    else:
        AIRS_cds = AIRS_cds.transpose(0,1,3,2)
        AIRS_cds_binned = AIRS_cds
    
    del AIRS_cds

    # 7. Flat Field Correction - use pre-loaded calibration data
    for i in range(CHUNKS_SIZE):
        if DO_FLAT:
            flat_airs = calibration_data[index_chunk[i]]['airs_flat']
            dead_airs = calibration_data[index_chunk[i]]['airs_dead']
            
            corrected_AIRS_cds_binned = correct_flat_field(flat_airs, dead_airs, AIRS_cds_binned[i])
            AIRS_cds_binned[i] = corrected_AIRS_cds_binned

    # Save data
    np.save(os.path.join(path_out, 'AIRS_clean_train_{}.npy'.format(n)), AIRS_cds_binned)
    
    del AIRS_cds_binned, calibration_data
    gc.collect()

100%|██████████| 275/275 [47:42<00:00, 10.41s/it]


In [10]:
TRAIN_CHUNKS_SIZE = CHUNKS_SIZE
CHUNKS_SIZE = 1
test_files = glob.glob(os.path.join(path_folder + 'test/', '*/*'))
test_index = get_index(test_files, CHUNKS_SIZE)

for n, index_chunk in enumerate(tqdm(test_index)):
    # Load all calibration data once at the beginning 
    calibration_data = load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, 'test')
    
    # Pre-allocate output arrays
    AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l), dtype=np.float32)
    
    # Parallel Processing
    # Determine number of workers (start with 2 to be safe)
    num_workers = min(2, CHUNKS_SIZE)
    
    # Prepare arguments for each observation
    args_list = []
    for i in range(CHUNKS_SIZE):
        args = (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK)
        args_list.append(args)
        
    # Process observations in parallel
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:        
        # Submit all tasks to the thread pool
        future_to_index = {executor.submit(process_single_observation_test, args): args[0] for args in args_list}
                
        # Collect results as they complete
        for future in tqdm(as_completed(future_to_index), total=CHUNKS_SIZE, desc=f"Processing observations"):
            i, airs_result = future.result()
            results.append((i, airs_result))
        
    # Sort results by observation index (i) to maintain order
    results.sort(key=lambda x: x[0])
    
    # Store results in existing arrays
    for result in results:
        i, airs_result = result
        AIRS_CH0_clean[i] = airs_result
        
    # 5. Get Correlated Double Sampling
    AIRS_cds = get_cds(AIRS_CH0_clean)
    
    del AIRS_CH0_clean
    
    # 6. (Optional) Time Binning (to reduce space)
    if TIME_BINNING:
        AIRS_cds_binned = bin_obs(AIRS_cds, binning=30)
    else:
        AIRS_cds = AIRS_cds.transpose(0,1,3,2)
        AIRS_cds_binned = AIRS_cds
    
    del AIRS_cds

    # 7. Flat Field Correction - use pre-loaded calibration data
    for i in range(CHUNKS_SIZE):
        if DO_FLAT:
            flat_airs = calibration_data[index_chunk[i]]['airs_flat']  
            dead_airs = calibration_data[index_chunk[i]]['airs_dead']
            
            corrected_AIRS_cds_binned = correct_flat_field(flat_airs, dead_airs, AIRS_cds_binned[i])
            AIRS_cds_binned[i] = corrected_AIRS_cds_binned

    # Save data
    np.save(os.path.join(path_out, 'AIRS_clean_test_{}.npy'.format(n)), AIRS_cds_binned)
    del AIRS_cds_binned, calibration_data

100%|██████████| 1/1 [00:03<00:00,  3.82s/it]


In [11]:
def load_data(file, chunk_size, nb_files): 
    data0 = np.load(file + '_0.npy')
    data_all = np.zeros((nb_files * chunk_size, data0.shape[1], data0.shape[2], data0.shape[3]))
    data_all[:chunk_size] = data0
    for i in range(1, nb_files): 
        data_all[i * chunk_size : (i+1) * chunk_size] = np.load(file + '_{}.npy'.format(i))
    return data_all 

data_train_AIRS = load_data(path_out + 'AIRS_clean_train', TRAIN_CHUNKS_SIZE, len(index)) 

print(data_train_AIRS.shape)


(1100, 187, 282, 32)


In [12]:
data_test_AIRS = load_data(path_out + 'AIRS_clean_test', CHUNKS_SIZE, 1) 

print(data_test_AIRS.shape)

(1, 187, 282, 32)


In [13]:
df_train = pd.read_csv(path_folder + 'train.csv')
df_train.set_index('planet_id', inplace=True)

planet_ids = np.concatenate(index)
df_train = df_train[df_train.index.isin(planet_ids)]

print(df_train.shape)
df_train.head()

(1100, 283)


,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,wl_11,wl_12,wl_13,wl_14,wl_15,wl_16,wl_17,wl_18,wl_19,wl_20,wl_21,wl_22,wl_23,wl_24,wl_25,wl_26,wl_27,wl_28,wl_29,wl_30,wl_31,wl_32,wl_33,wl_34,wl_35,wl_36,wl_37,wl_38,wl_39,wl_40,wl_41,wl_42,wl_43,wl_44,wl_45,wl_46,wl_47,wl_48,wl_49,wl_50,wl_51,wl_52,wl_53,wl_54,wl_55,wl_56,wl_57,wl_58,wl_59,wl_60,wl_61,wl_62,wl_63,wl_64,wl_65,wl_66,wl_67,wl_68,wl_69,wl_70,wl_71,wl_72,wl_73,wl_74,wl_75,wl_76,wl_77,wl_78,wl_79,wl_80,wl_81,wl_82,wl_83,wl_84,wl_85,wl_86,wl_87,wl_88,wl_89,wl_90,wl_91,wl_92,wl_93,wl_94,wl_95,wl_96,wl_97,wl_98,wl_99,wl_100,wl_101,wl_102,wl_103,wl_104,wl_105,wl_106,wl_107,wl_108,wl_109,wl_110,wl_111,wl_112,wl_113,wl_114,wl_115,wl_116,wl_117,wl_118,wl_119,wl_120,wl_121,wl_122,wl_123,wl_124,wl_125,wl_126,wl_127,wl_128,wl_129,wl_130,wl_131,wl_132,wl_133,wl_134,wl_135,wl_136,wl_137,wl_138,wl_139,wl_140,wl_141,wl_142,wl_143,wl_144,wl_145,wl_146,wl_147,wl_148,wl_149,wl_150,wl_151,wl_152,wl_153,wl_154,wl_155,wl_156,wl_157,wl_158,wl_159,wl_160,wl_161,wl_162,wl_163,wl_164,wl_165,wl_166,wl_167,wl_168,wl_169,wl_170,wl_171,wl_172,wl_173,wl_174,wl_175,wl_176,wl_177,wl_178,wl_179,wl_180,wl_181,wl_182,wl_183,wl_184,wl_185,wl_186,wl_187,wl_188,wl_189,wl_190,wl_191,wl_192,wl_193,wl_194,wl_195,wl_196,wl_197,wl_198,wl_199,wl_200,wl_201,wl_202,wl_203,wl_204,wl_205,wl_206,wl_207,wl_208,wl_209,wl_210,wl_211,wl_212,wl_213,wl_214,wl_215,wl_216,wl_217,wl_218,wl_219,wl_220,wl_221,wl_222,wl_223,wl_224,wl_225,wl_226,wl_227,wl_228,wl_229,wl_230,wl_231,wl_232,wl_233,wl_234,wl_235,wl_236,wl_237,wl_238,wl_239,wl_240,wl_241,wl_242,wl_243,wl_244,wl_245,wl_246,wl_247,wl_248,wl_249,wl_250,wl_251,wl_252,wl_253,wl_254,wl_255,wl_256,wl_257,wl_258,wl_259,wl_260,wl_261,wl_262,wl_263,wl_264,wl_265,wl_266,wl_267,wl_268,wl_269,wl_270,wl_271,wl_272,wl_273,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
planet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
34983,0.018291,0.018088,0.018087,0.018085,0.018084,0.018084,0.018084,0.018084,0.018085,0.018084,0.018083,0.018080,0.018076,0.018072,0.018068,0.018063,0.018060,0.018060,0.018062,0.018069,0.018079,0.018091,0.018103,0.018111,0.018115,0.018117,0.018119,0.018123,0.018129,0.018136,0.018139,0.018137,0.018133,0.018131,0.018131,0.018132,0.018133,0.018134,0.018135,0.018136,0.018137,0.018136,0.018134,0.018133,0.018132,0.018132,0.018131,0.018129,0.018127,0.018126,0.018126,0.018126,0.018124,0.018119,0.018115,0.018111,0.018108,0.018106,0.018104,0.018102,0.018100,0.018098,0.018096,0.018095,0.018095,0.018095,0.018095,0.018094,0.018092,0.018090,0.018089,0.018088,0.018087,0.018084,0.018082,0.018080,0.018079,0.018079,0.018080,0.018080,0.018079,0.018076,0.018074,0.018073,0.018076,0.018082,0.018093,0.018105,0.018116,0.018124,0.018130,0.018134,0.018136,0.018136,0.018136,0.018137,0.018137,0.018137,0.018138,0.018140,0.018144,0.018147,0.018148,0.018148,0.018150,0.018152,0.018154,0.018156,0.018157,0.018157,0.018158,0.018158,0.018158,0.018157,0.018158,0.018159,0.018160,0.018161,0.018162,0.018163,0.018164,0.018167,0.018169,0.018171,0.018171,0.018171,0.018170,0.018170,0.018169,0.018168,0.018167,0.018167,0.018169,0.018173,0.018174,0.018171,0.018167,0.018165,0.018164,0.018164,0.018164,0.018163,0.018162,0.018162,0.018161,0.018160,0.018159,0.018160,0.018161,0.018163,0.018165,0.018166,0.018168,0.018171,0.018173,0.018174,0.018176,0.018178,0.018178,0.018175,0.018173,0.018171,0.018170,0.018169,0.018168,0.018166,0.018165,0.018165,0.018163,0.018159,0.018155,0.018154,0.018152,0.018148,0.018144,0.018142,0.018142,0.018146,0.018155,0.018172,0.018192,0.018207,0.018211,0.018208,0.018200,0.018190,0.018179,0.018169,0.018161,0.018156,0.018152,0.018150,0.018147,0.018147,0.018150,0.018154,0.018156,0.018157,0.018157,0.018157,0.018156,0.018155,0.018155,

## Split into train and validation sets

In [14]:
# Simple 80/20 split to start - no shuffling
n = round(.8 * len(df_train))

train_AIRS = data_train_AIRS[:n]
val_AIRS = data_train_AIRS[n:]
print(len(train_AIRS), len(val_AIRS))

train_labels = df_train.iloc[:n,:] 
val_labels = df_train.iloc[n:,:]
print(train_labels.shape, val_labels.shape)

del data_train_AIRS

880 220
(880, 283) (220, 283)


## Define the model

In [15]:
inputs = Input(shape=(375, 282, 32), name='inputs')
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)

# Two output heads
mean_output = Dense(283, activation='linear', name='mean')(x)  
log_std_output = Dense(283, activation='linear', name='log_std')(x) 
std_output = Lambda(lambda x: tf.exp(0.5 * x), name='std')(log_std_output)

# Concatenate outputs for submission
outputs = Concatenate(name='outputs')([mean_output, std_output])

model = Model(inputs=inputs, outputs=outputs)
model.summary()

I0000 00:00:1759207665.923259      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1759207665.926191      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 375, 282,  │          0 │ -                 │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 373, 280,  │      9,248 │ inputs[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 186, 140,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 184, 138,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 92, 69,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 90, 67,    │     36,928 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 385920)    │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │ 24,698,944 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_std (Dense)     │ (None, 283)       │     18,395 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Dense)        │ (None, 283)       │     18,395 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ std (Lambda)        │ (None, 283)       │          0 │ log_std[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ outputs             │ (None, 566)       │          0 │ mean[0][0],       │
│ (Concatenate)       │                   │            │ std[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,800,406 (94.61 MB)

 Trainable params: 24,800,406 (94.61 MB)

 Non-trainable params: 0 (0.00 B)

## Compile and train the model

In [16]:
def nll_loss(y_true, y_pred):
    mu, std = y_pred[:, :283], y_pred[:, 283:]
    return tf.reduce_mean(0.5 * tf.math.log(2 * np.pi * std**2) + 0.5 * ((y_true - mu)**2 / std**2))

model.compile(optimizer = 'adam', loss = nll_loss)

In [ ]:
model.fit(train_AIRS, train_labels.values, 
          validation_data=(val_AIRS, val_labels.values),
          epochs=10, batch_size=8, verbose=0)

del train_AIRS, val_AIRS
del train_labels, val_labels
gc.collect()

## Generate predictions

In [ ]:
gc.collect()

In [ ]:
# Generate predictions
predictions = model.predict(data_test_AIRS)  
means = predictions[:, :283]  
stds = predictions[:, 283:] 
stds = np.abs(stds)

# Create submission DataFrame
# Fix test_index if it's nested from chunking
if isinstance(test_index, list) and len(test_index) > 0:
    test_index_flat = np.concatenate(test_index)
else:
    test_index_flat = test_index
    
# Load sample submission 
df_sample = pd.read_csv(path_folder + 'sample_submission.csv')

# Create submission DataFrame matching sample format exactly
df_submission = df_sample.copy()
df_submission['planet_id'] = test_index_flat

df_submission.iloc[:, 1:284] = means.astype(np.float32)  
df_submission.iloc[:, 284:567] = stds.astype(np.float32)   

# Replace inf and NaN values
df_submission = df_submission.replace([np.inf, -np.inf], np.nan)
df_submission = df_submission.fillna(0.0)

# Ensure all columns have correct data types (skip planet_id)
for col in df_submission.columns[1:]: 
    df_submission[col] = df_submission[col].astype(np.float32)

In [ ]:
df_submission.to_csv('/kaggle/working/submission.csv', index=False)